In [51]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [74]:
# assign the driver path
driver_path = './chromedriver'

In [75]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

In [76]:
# create a driver object using driver_path as a parameter
driver = webdriver.Chrome(options = options, service = Service(executable_path=driver_path)) # run in background
# driver = webdriver.Chrome(service = Service(executable_path=driver_path)) # show browser

In [77]:
# assign your website to scrape
web = 'https://www.amazon.com'

driver.get(web)

In [78]:
# assign any keyword for searching
keyword = '眼鏡'

In [79]:
# create WebElement for a search box
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')

In [80]:
# type the keyword in searchbox
search_box.send_keys(keyword)

In [81]:
# create WebElement for a search button 
search_button = driver.find_element(By.ID, 'nav-search-submit-button')

In [82]:
# click search_button
search_button.click()

In [83]:
# wait for the page to download
driver.implicitly_wait(5)

In [84]:
# create empty lists for containing the data we'd like to scrape
product_name = []
product_asin = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []
results = []

In [85]:
items = driver.find_elements(By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')

In [86]:
items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))

In [87]:
for item in items:
    # find product name
    name = item.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]')
#     product_name.append(name.text)
    
    # find product asin
    data_asin = item.get_attribute("data-asin")
#     product_asin.append(data_asin)
    
    
    # find prices
    whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
    fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

    if whole_price != [] and fraction_price != []:
        price = '.'.join([whole_price[0].text, fraction_price[0].text])
    else:
        price = 0
        
#     product_price.append(price)
    
    # find a ratings box
    ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

    if ratings_box != []:
        ratings = ratings_box[0].get_attribute('aria-label')
        ratings_num = ratings_box[1].get_attribute('aria-label')
    else:
        ratings, ratings_num = 0, 0

#     product_ratings.append(ratings)
#     product_ratings_num.append(str(ratings_num))

    # find the details link
    link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
    
    # find the product image link
    img_link = item.find_element(By.XPATH, './/img[@class="s-image"]').get_attribute('src')
    
#     product_link.append(link)
    results.append((name.text, data_asin, price, ratings, str(ratings_num), link, img_link))

In [88]:
# quit the driver after finishing scraping 
driver.quit()

In [89]:
df = pd.DataFrame(results, columns=["product_name", "product_asin", "product_price", "product_ratings", "product_ratings_num", "product_link", "img_link"])
print(df)

                                         product_name product_asin  \
0   Shiratori New Vintage Fashion Half Frame Semi-...   B01CYK3CCG   
1   FONHCOO Blue Light Blocking Glasses Women Men ...   B08G8N1455   
2          Eyeglasses Michael Kors MK 4074 3050 Clear   B08F24J4BG   
3   MOLDERP Glasses Ear Grip - Eyeglasses Temple T...   B07B6K9SZQ   
4   Nannini Compact One Optics 1.0 Temples Reading...   B006DY69KM   
5                           Glass (The Crank Trilogy)   1442471824   
6   Pro Acme Classic Round Metal Clear Lens Glasse...   B01N5DIYWI   
7   Versace VE 1175B Eyeglasses w/Gold Frame and N...   B0047H2DP2   
8   Christopher Cloos - Passable - Danish Design B...   B08G562XCK   
9   Echo Frames (2nd Gen) | Smart audio glasses wi...   B083C58VDP   
10  SomniLight FL-41 Light Sensitivity Glasses for...   B083VPDTSV   
11  Gucci Women's Gg0025o 56Mm Optical Glasses, Bl...   B06XCRH1J6   
12  Music Glasses Blue Light Blocking Metal Frame ...   B097RRJ9DQ   
13  Cyxus Round Cat 

In [90]:
df.to_excel("test.xlsx", sheet_name="test", index=False)